<a href="https://colab.research.google.com/github/babypotatotang/Introduction-to-DeepLearning/blob/main/8.%20NLP%20Preprocess/03.Torchtext%20Tutorial_Kor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 형태소 분석기 Mecab 설치**

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

# **2. 훈련 데이터와 테스트 데이터로 다운로드하기**

In [ ]:
import urllib.request 
import pandas as pd 

In [ ]:
# 영화 리뷰 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
# 훈련데이터와 테스트 데이터 각각 정의
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# 각각 id, document, label이라는 3개의 열로 구성되어 있으며 각각 리뷰, 리뷰에 대한 감성 레이블로 이루어져있음. 
print(f'훈련 데이터 샘플의 개수: {len(train_df)}') # 15만
print(f'테스트 데이터 샘플의 개수 : {len(test_df)}') # 5만

# **3. 필드 정의하기 (torchtext.data)**

In [ ]:
# 0.10.0 버전으로 install 
!pip install torchtext==0.10.0

In [ ]:
from torchtext.legacy import data # torchtext.data 임포트
from konlpy.tag import Mecab

In [ ]:
# Mecab을 토크나이저로 사용
tokenizer = Mecab()

In [ ]:
# 필드 정의 
ID = data.Field(sequential = False, 
                use_vocab = False)  # 데이터 프로세싱 과정에서 실제로 사용하지 않음 
TEXT = data.Field(sequential = True, 
                  use_vocab = True, tokenize = tokenizer.morphs, lower = True, batch_first = True, 
                  fix_length = 20)
LABEL = data.Field(sequential = False, use_vocab = True)

# **4. 데이터셋 만들기**


In [ ]:
from torchtext.legacy.data import TabularDataset

In [ ]:
train_data, test_data = TabularDataset.splits(
        path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

In [ ]:
print(f'훈련 샘플의 개수 : {len(train_data)}')
print(f'테스트 샘플의 개수 : {len(test_data)}')

In [ ]:
print(vars(train_data[0]))

# **5. 단어 집합(Vocabulary) 만들기**


In [ ]:
# 단어 집합 생성 
TEXT.build_vocab(train_data, min_freq = 10, max_size = 10000)

In [ ]:
print(f'단어 집합의 크기 : {len(TEXT.vocab)}')

In [ ]:
print(TEXT.vocab.stoi)

# **6. 토치텍스트의 데이터로더 만들기**

In [ ]:
from torchtext.legacy.data import Iterator

In [ ]:
batch_size = 5

In [ ]:
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

In [ ]:
# 배치 크기 5씩 묶어주었으므로, 미니배치 수는 30,000개, 10,000개 
print(f'훈련 데이터의 미니 배치 수 : {len(train_loader)}')
print(f'테스트 데이터의 미니 배치 수 : {len(test_loader)}')

In [ ]:
batch = next(iter(train_loader)) # 첫번째 미니 배치

* 배치 크기가 5이기 때문에 5개의 샘플이 출력됨 
* 각 샘플의 길이는 20(`fix_length`)를 가지며, 미니 배치의 크기는 5 x 20 (배치 크기 x 샘플의 길이) 임.  